Principal Files
This data challenge contains one dataset of 2000 training sequences. The main files available are the following ones

* Xtr.csv - the training sequences.
* 
* Xte.csv - the test sequences.
* 
* Ytr.csv - the sequence labels of the training sequences indicating bound (1) or not (0).
* 
Each row of Xtr.csv represents a sequence. Xte.csv contains 1000 test sequences, for which you need to predict. Ytr.csv contains the labels corresponding to the training data, in the same format as a submission file.

In [1]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
import optuna
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score

import os


In [2]:

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/kernel-methods-ammi-2020/Xtr_mat100.csv
/kaggle/input/kernel-methods-ammi-2020/Xte.csv
/kaggle/input/kernel-methods-ammi-2020/Xte_mat100.csv
/kaggle/input/kernel-methods-ammi-2020/Ytr.csv
/kaggle/input/kernel-methods-ammi-2020/Xtr.csv


In [3]:
train_data=pd.read_csv("/kaggle/input/kernel-methods-ammi-2020/Xtr.csv", sep=' ',index_col=0).values

labels=pd.read_csv("/kaggle/input/kernel-methods-ammi-2020/Ytr.csv", sep=',',index_col=0).values


test_data=pd.read_csv("/kaggle/input/kernel-methods-ammi-2020/Xte.csv",sep=' ', index_col=0).values


#optional data 

train_op_data=pd.read_csv("/kaggle/input/kernel-methods-ammi-2020/Xtr_mat100.csv", sep=' ',header=None).values
test_op_data=pd.read_csv("/kaggle/input/kernel-methods-ammi-2020/Xte_mat100.csv", sep=' ',header=None).values





## Change the labels from 0,1 to -1 ,1

In [4]:
labels= np.where(labels==0,-1,1)

train_op_data[:5,:5]

array([[0.01086957, 0.01086957, 0.01086957, 0.0326087 , 0.01086957],
       [0.        , 0.01086957, 0.02173913, 0.        , 0.02173913],
       [0.02173913, 0.        , 0.        , 0.        , 0.04347826],
       [0.        , 0.02173913, 0.0326087 , 0.        , 0.02173913],
       [0.01086957, 0.04347826, 0.        , 0.0326087 , 0.01086957]])

In [5]:
labels

array([[ 1],
       [-1],
       [ 1],
       ...,
       [ 1],
       [ 1],
       [ 1]])

# Data exploring

In [6]:
print(train_data.shape,test_data.shape)

(2000, 0) (1000, 0)


## optional data view

In [7]:
print(train_op_data.shape)

(2000, 100)


# Split the training dataset in train and validation set

In [8]:

X_train_mat100 = scale(train_op_data)
# 
X_train, X_val, y_train, y_val = train_test_split(
    X_train_mat100, labels, test_size=0.33, random_state=42)

print(X_train.shape,X_val.shape,y_train.shape, y_val.shape)


(1340, 100) (660, 100) (1340, 1) (660, 1)


# soft margin svm with SGD from scratch

In [24]:
class SVM:
    def __init__(self, lr,lamb, epoch):
        self.lr=lr
        self.lamb=lamb
        self.epoch=epoch
    def compute_cost(self,W, X, Y):
        # calculate hinge loss
        N = X.shape[0]
        
        distances = 1 - Y * (np.dot(X, W))
        distances[distances < 0] = 0  # equivalent to max(0, distance)
        hinge_loss = self.lamb * (np.sum(distances) / N)

        # calculate cost
        cost = 1 / 2 * (np.dot(W, W)) + hinge_loss
        return cost
    
    def calculate_cost_gradient(self,W, X_batch, Y_batch):
        # if only one example is passed (eg. in case of SGD)
        if type(Y_batch) == np.float64:
            Y_batch = np.array([Y_batch])
            X_batch = np.array([X_batch])  # gives multidimensional array

        distance = 1 - (Y_batch * np.dot(X_batch, W))
        dw = np.zeros(len(W))

        for ind, d in enumerate(distance):
            if max(0, d) == 0:
                di = W
            else:
                di = W -(self.lamb* Y_batch[ind] * X_batch[ind])
            dw += di

        dw = dw/len(Y_batch)  # average
        return dw
    
    def sgd(self,features, outputs):
        
        weights = np.zeros(features.shape[1])
        nth = 0
        prev_cost = float("inf")
        cost_threshold = 2  # in percent
        # stochastic gradient descent
        for epoch in range(1, self.epoch):
            # shuffle to prevent repeating update cycles
            X, Y = (features, outputs)
            for ind, x in enumerate(X):
                ascent = self.calculate_cost_gradient(weights, x, Y[ind])
                weights = weights -(self.lr * ascent)


            cost = self.compute_cost(weights, features, outputs)
#             print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
        return weights
    
    
    
    def train_func(self,x_train,y_train):
        W = self.sgd(x_train, y_train)
#         print("training finished.")
        return W

    def validation(self,x_val,y_val,w):
        y_test_predicted = np.array([])
        for i in range(x_val.shape[0]):
            yp = np.sign(np.dot(w, x_val[i])) #model
            y_test_predicted = np.append(y_test_predicted, yp)
        return accuracy_score(y_val, y_test_predicted)



# Insert the bias

In [25]:
def insert_intercept(train_op_data):
    N=train_op_data.shape[0]
   
    a = np.ones((N,1))
    
    train_op_data=np.append(train_op_data,a, axis=1)
    return train_op_data

## Test the model

In [26]:
#train_op_data=insert_intercept(train_op_data)
svm_sgd=SVM(lr=0.1,lamb=1,epoch=10)

In [27]:
w=svm_sgd.train_func(X_train,y_train)
svm_sgd.validation(X_val,y_val,w)

0.5257575757575758

## cross validation

In [28]:

def cross_validate(x_data,y_data,model,lr,lamda=0.2,epoch=10,k=5):
    if len(x_data)%k != 0:
        print('cant vsplit',len(x_data),' by ',k)
        return
    
    x_data_splitted = np.vsplit(x_data,k)
    y_data_splitted = np.vsplit(y_data,k)
    
    aggrigate_result = []
    
    for i in range(len(x_data_splitted)):
        train = []
        test = []
        items = [j for j in range(len(x_data_splitted)) if j !=i ]
        x_test = x_data_splitted[i]
        y_test = y_data_splitted[i]
        
        for item in items:
            if len(train) == 0:
                x_train = x_data_splitted[item]
                y_train = y_data_splitted[item]
                
                
            else:
                x_train = np.concatenate((x_train,x_data_splitted[item]), axis=0)
                
                
                y_train = np.concatenate((y_train,y_data_splitted[item]), axis=0)
                
                
       
        w=model.train_func(x_train,y_train)
       
       
        
        result = model.validation(x_test,y_test,w)
        aggrigate_result.append(result)
         
        
        value = sum(aggrigate_result)/len(aggrigate_result)
        
        
    return value 

## Hyperparameter search using optuna

In [31]:
def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    lamda = trial.suggest_loguniform('lamda', 0.2, 1)
    k =  trial.suggest_categorical('k', [4,5,8,10])
    epoch =  trial.suggest_int('epoch', 3, 15)
    return cross_validate(train_op_data, labels,svm_sgd,lr=lr,lamda=lamda,k=k,epoch=epoch)

In [32]:
import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective, n_trials=200,show_progress_bar=True)

[I 2020-05-26 10:01:48,758] Finished trial#0 with value: 0.4999999999999999 with parameters: {'lr': 3.156669340679139e-05, 'lamda': 0.20278061256132357, 'k': 10, 'epoch': 14}. Best is trial#0 with value: 0.4999999999999999.
[I 2020-05-26 10:01:49,044] Finished trial#1 with value: 0.4999999999999999 with parameters: {'lr': 0.03224239746196466, 'lamda': 0.49505289735630237, 'k': 10, 'epoch': 13}. Best is trial#0 with value: 0.4999999999999999.
[I 2020-05-26 10:01:49,345] Finished trial#2 with value: 0.505 with parameters: {'lr': 0.002259097170901521, 'lamda': 0.20838394504806315, 'k': 4, 'epoch': 4}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:01:49,642] Finished trial#3 with value: 0.4999999999999999 with parameters: {'lr': 3.5688339150237606e-05, 'lamda': 0.5867656348476377, 'k': 10, 'epoch': 7}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:01:49,941] Finished trial#4 with value: 0.5 with parameters: {'lr': 0.0017176422607032866, 'lamda': 0.5096123664424019, 'k': 5, 'epo

[I 2020-05-26 10:02:01,217] Finished trial#41 with value: 0.505 with parameters: {'lr': 1.066348925208264e-05, 'lamda': 0.8919981250959458, 'k': 8, 'epoch': 5}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:01,521] Finished trial#42 with value: 0.505 with parameters: {'lr': 1.5775937133762342e-05, 'lamda': 0.9958940234318522, 'k': 8, 'epoch': 8}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:01,824] Finished trial#43 with value: 0.505 with parameters: {'lr': 1.6689308809363798e-05, 'lamda': 0.8893810342552694, 'k': 8, 'epoch': 8}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:02,116] Finished trial#44 with value: 0.505 with parameters: {'lr': 5.743250168327003e-05, 'lamda': 0.5495220600975191, 'k': 8, 'epoch': 3}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:02,416] Finished trial#45 with value: 0.505 with parameters: {'lr': 1.964636760838781e-05, 'lamda': 0.836164228003733, 'k': 8, 'epoch': 8}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02

[I 2020-05-26 10:02:13,672] Finished trial#82 with value: 0.505 with parameters: {'lr': 1.5289137849486307e-05, 'lamda': 0.7863763571785314, 'k': 8, 'epoch': 8}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:13,967] Finished trial#83 with value: 0.505 with parameters: {'lr': 0.010964130241952178, 'lamda': 0.6579402486305578, 'k': 8, 'epoch': 14}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:14,267] Finished trial#84 with value: 0.505 with parameters: {'lr': 0.08698219883216297, 'lamda': 0.4916908205210452, 'k': 8, 'epoch': 15}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:14,566] Finished trial#85 with value: 0.505 with parameters: {'lr': 0.06453281258163607, 'lamda': 0.5542847104070878, 'k': 8, 'epoch': 6}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:14,857] Finished trial#86 with value: 0.505 with parameters: {'lr': 0.044720613754619186, 'lamda': 0.5216359352838875, 'k': 8, 'epoch': 14}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:15

[I 2020-05-26 10:02:26,157] Finished trial#123 with value: 0.505 with parameters: {'lr': 0.028951077153470374, 'lamda': 0.5668276361304311, 'k': 8, 'epoch': 14}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:26,455] Finished trial#124 with value: 0.505 with parameters: {'lr': 0.03934808889931656, 'lamda': 0.48466040895281676, 'k': 8, 'epoch': 15}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:26,757] Finished trial#125 with value: 0.505 with parameters: {'lr': 0.02140002782854367, 'lamda': 0.45539679883545175, 'k': 8, 'epoch': 4}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:27,056] Finished trial#126 with value: 0.505 with parameters: {'lr': 0.08088183943439811, 'lamda': 0.584341950948867, 'k': 8, 'epoch': 15}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:27,356] Finished trial#127 with value: 0.505 with parameters: {'lr': 0.011863595796069657, 'lamda': 0.5342377861048171, 'k': 8, 'epoch': 6}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02

[I 2020-05-26 10:02:38,975] Finished trial#164 with value: 0.505 with parameters: {'lr': 1.2374582724527737e-05, 'lamda': 0.9534599306912925, 'k': 4, 'epoch': 11}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:39,292] Finished trial#165 with value: 0.5 with parameters: {'lr': 1.4362213399937621e-05, 'lamda': 0.612285853950937, 'k': 5, 'epoch': 8}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:39,620] Finished trial#166 with value: 0.505 with parameters: {'lr': 0.0003603728414594243, 'lamda': 0.39847871659302503, 'k': 4, 'epoch': 10}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:39,933] Finished trial#167 with value: 0.505 with parameters: {'lr': 0.008546903005037448, 'lamda': 0.6604930067323166, 'k': 8, 'epoch': 13}. Best is trial#2 with value: 0.505.
[I 2020-05-26 10:02:40,236] Finished trial#168 with value: 0.505 with parameters: {'lr': 0.0052161453646496925, 'lamda': 0.5357487103455103, 'k': 8, 'epoch': 13}. Best is trial#2 with value: 0.505.
[I 2020-05-2

In [36]:
svm_sgd=SVM(lr=0.0166431449930997,lamb=0.5963218576040026,epoch=10)

In [37]:
w=svm_sgd.train_func(X_train,y_train)
svm_sgd.validation(X_val,y_val,w)

0.47424242424242424

In [41]:
cross_validate(train_op_data,labels,svm_sgd,lr=0.01,lamda=0.2,epoch=100,k=5)

0.501

In [44]:
!pip3 install  qpsolvers
from qpsolvers import solve_qp

  Created wheel for qpsolvers: filename=qpsolvers-1.3-py3-none-any.whl size=17440 sha256=b8a569047373f299bb540ee1907a773f561d596d1acde544240d47657c8fd920
  Stored in directory: /root/.cache/pip/wheels/52/81/13/36e48a1e0441f4a9a22e626b477b58604eb33c1d6bdd14a265
  Created wheel for quadprog: filename=quadprog-0.1.7-cp37-cp37m-linux_x86_64.whl size=329595 sha256=194a4d0687f71cb35cb394e8f669d3662639bf4ea0317752606cccea2a41343d
  Stored in directory: /root/.cache/pip/wheels/70/93/15/32155be29d7e221844ba69884ab5d8c9e36998d6d4a92cd62a
Successfully built qpsolvers quadprog
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:

# def svm_primal_hard_to_qp(X, y):
#     n, p = X.shape
#     assert (len(y) == n)
    
#     Xy = np.diag(y).dot(X)
#     # Primal formulation, hard margin
#     diag_P = np.ones(p + 1) # correct this!
#     diag_P[-1] = 0 # The one multiplies with b
#     # As a regularization, we add epsilon * identity to P
#     eps = 1e-12
#     diag_P += eps
#     P = np.diag(diag_P)
    
#     q = np.zeros(p + 1)
#     G = np.hstack([Xy, y[:, np.newaxis]]) # newaxis transforms y into column matrix
#     # G = np.vstack([Xy.T, y]).T
#     h = - np.ones(n)
#     A = None
#     b = None
    
#     return P, q, G, h, A, b

# x = solve_qp(*svm_primal_hard_to_qp(x_train, y_train))
# n, p = X_train.shape
# w, b = x[:-1], x[-1]

In [34]:
!pip install cvxopt

     |████████████████████████████████| 11.6 MB 383 kB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [35]:
import cvxopt

# SVM dual problem

In [53]:

cvxopt.solvers.options['show_progress'] = False


def getDual(X_train,y_train):
    m,n = X_train.shape
    y = y_train.reshape(-1,1) * 1.
    X_dash = y_train * X_train
    H = np.dot(X_dash , X_dash.T) * 1
   # P = P = cvxopt_matrix(H)
  
    #Converting into cvxopt format
    P = cvxopt.matrix(H)
    q = cvxopt.matrix(-np.ones((m, 1)))
    G = cvxopt.matrix(-np.eye(m))
    h = cvxopt.matrix(np.zeros(m))
    A = cvxopt.matrix(y.reshape(1, -1))
    b = cvxopt.matrix(np.zeros(1))

    #Setting solver parameters (change default to decrease tolerance) 
    cvxopt.solvers.options['show_progress'] = False
    cvxopt.solvers.options['abstol'] = 1e-10
    cvxopt.solvers.options['reltol'] = 1e-10
    cvxopt.solvers.options['feastol'] = 1e-10

    #Run solver
    sol = cvxopt.solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x'])
    return alphas
        

In [76]:
#w parameter in vectorized form
alphas=getDual(X_train,y_train)
w = np.sum((y_train* alphas) * X_train, axis=0)


S = (alphas > 1e-4).reshape(-1)

#Computing b
b = y_train[S] - np.dot(X_train[S], w)
print(b)
b=b[0]
print(b)


[[ 1.00006208  1.00025909  0.99968729 ...  1.0003954   1.0000324
   1.00119241]
 [-0.99993792 -0.99974091 -1.00031271 ... -0.9996046  -0.9999676
  -0.99880759]
 [-0.99993792 -0.99974091 -1.00031271 ... -0.9996046  -0.9999676
  -0.99880759]
 ...
 [-0.99993792 -0.99974091 -1.00031271 ... -0.9996046  -0.9999676
  -0.99880759]
 [ 1.00006208  1.00025909  0.99968729 ...  1.0003954   1.0000324
   1.00119241]
 [ 1.00006208  1.00025909  0.99968729 ...  1.0003954   1.0000324
   1.00119241]]
[1.00006208 1.00025909 0.99968729 ... 1.0003954  1.0000324  1.00119241]


In [77]:
len(b)

1340

In [84]:
def validation(x_val,y_val,w,b):
        y_test_predicted = np.array([])
        for i in range(x_val.shape[0]):
            #print(w.shape,x_val.shape)
            yp = np.sign(np.dot(w.T, x_val[i])+b[i]) #model
            y_test_predicted = np.append(y_test_predicted, yp)
            print(y_test_predicted.min())
        return accuracy_score(y_val, y_test_predicted)

In [85]:
validation(X_val,y_val,w,b)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


0.5287878787878788

# # soft margin SVM with kernel

In [63]:
from numpy import linalg
import cvxopt
import cvxopt.solvers

# Define a set of kernels

In [64]:

def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=2):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))


kernels=[linear_kernel,polynomial_kernel,gaussian_kernel]

In [95]:
class SVM(object):

    def __init__(self, kernel=linear_kernel, C=0.0001):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)

    def fit(self, X, y):
        n_samples, n_features = X.shape

        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])
        y = y_train.reshape(-1,1) * 1.
        X_dash = y_train * X_train
        H = np.dot(X_dash , X_dash.T) * 1

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        b = cvxopt.matrix(0.0)

        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))

        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        a = np.ravel(solution['x'])

        sv = a > 1e-5
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == linear_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else:
            self.w = None

    def project(self, X):
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                y_predict[i] = s
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [96]:
for k in kernels:
    #print(k)
    svmm=SVM(kernel=k)
    svmm.fit(X_train,y_train)
    y_predict = svmm.predict(X_val)
    print(y_predict)
    
    print(accuracy_score(y_val, y_predict))


1263 support vectors out of 1340 points
[-1. -1.  1. -1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.  1.
 -1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1.  1.
  1.  1.  1. -1.  1.  1. -1.  1. -1. -1.  1. -1.  1.  1.  1. -1.  1.  1.
  1.  1. -1. -1.  1.  1.  1.  1. -1.  1. -1. -1.  1.  1. -1.  1. -1.  1.
  1.  1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.
  1.  1.  1. -1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1.  1.
 -1. -1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1.
 -1. -1.  1.  1.  1.  1. -1.  1.  1. -1. -1. -1. -1. -1. -1.  1.  1.  1.
  1. -1.  1.  1. -1.  1. -1.  1.  1.  1.  1. -1. -1. -1. -1.  1. -1.  1.
 -1.  1.  1. -1.  1. -1.  1. -1.  1. -1.  1.  1. -1.  1.  1. -1.  1.  1.
  1. -1. -1. -1.  1.  1.  1.  1.  1. -1. -1.  1. -1.  1.  1.  1.  1.  1.
  1.  1. -1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.
 -1. -1.  1. -1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1.
 -1.  1.  1